# Creating an assignment and rubric on Canvas with the API

These tools were primarily developed to support my instruction of the University of British Columbia's Master of Data Science program.

In [1]:
import json
import os
from datetime import datetime, timedelta
from canvasapi import Canvas

## Creating Assignment

In [ ]:
# UBC credentials
# API_URL = "https://canvas.ubc.ca/"        # default is canvas.ubc
# API_KEY = os.getenv("CANVAS_API")         # canvas.ubc instructor token
# COURSE_CODE = 53659                       # canvas 511 course code

# Canvas Instructure Credentials
API_URL = "https://canvas.instructure.com/" # default is canvas.ubc
API_KEY = os.getenv("CANVAS_API_TEST_I")    # canvas.instructure instructor token
COURSE_CODE = 2313167                       # canvas instructure course code

In [ ]:
canvas = Canvas(API_URL, API_KEY)
course = canvas.get_course(COURSE_CODE)

# our assignments are due the saturday after they are released
next_saturday = datetime.now() + timedelta((5 - datetime.now().weekday()) % 7) # the 5 is "Saturday", 0 would be "Monday"
assignment_due_date = datetime(year=next_saturday.year, month=next_saturday.month, day=next_saturday.day, hour=18, minute=5)            

# create assignment
new_assignment = course.create_assignment({
    'name': 'Test Assignment',
    'description': 'This is a test assignment',
    'submission_types': ['online_upload'],
    'allowed_extensions': ['html'],
    'published': True,
    'due_at': assignment_due_date
})

In [ ]:
# close assignment
# new_assignment = new_assignment.edit(assignment={'due_at': datetime.datetime.now()})  # allows late submissions
# new_assignment = new_assignment.edit(assignment={'lock_at': datetime.datetime.now()})  # no late submissions

In [ ]:
# delete assignment
# new_assignment.delete();

## Creating a Rubric for the assignment

#### Load weights.json

- We generate .json file that contain the rubric for grading our assignments
- They look like this:

In [2]:
with open('example_data/weights.json') as f:
    weights = json.load(f)
weights

{'Instructions': {'Mechanics': 2, 'Spark (bonus)': 1},
 '1.1': {'Autograde': 1, 'Spark (bonus)': 1},
 '1.2': {'Autograde': 1, 'Spark (bonus)': 1},
 '1.3': {'Autograde': 1, 'Spark (bonus)': 1},
 '1.4': {'Autograde': 1, 'Spark (bonus)': 1},
 '2.1': {'Accuracy': 1, 'Spark (bonus)': 1},
 '2.2': {'Accuracy': 1, 'Spark (bonus)': 1},
 '2.3': {'Accuracy': 1, 'Spark (bonus)': 1},
 '3.1': {'Autograde': 1, 'Spark (bonus)': 1},
 '3.2': {'Autograde': 1, 'Spark (bonus)': 1},
 'Exercise 4: Making Sushi': {'Autograde': 1, 'Spark (bonus)': 1},
 '5.1': {'Autograde': 1, 'Spark (bonus)': 1},
 '5.2': {'Autograde': 1, 'Spark (bonus)': 1},
 '5.3': {'Autograde': 1, 'Spark (bonus)': 1},
 '6.1': {'Accuracy': 1, 'Quality': 1, 'Spark (bonus)': 1},
 '6.2': {'Accuracy': 2, 'Quality': 2, 'Spark (bonus)': 1},
 'Exercise 7: Simulating a Random Walk in 2D': {'Accuracy': 3,
  'Quality': 3,
  'Spark (bonus)': 1},
 '(Optional) Exercise 8: Integer Division': {'Reasoning': 1,
  'Spark (bonus)': 1,
  'is_bonus': True}}

In [3]:
# These are hard-wired grade ranges
UBC_LETTER_GRADES = {
    (90,100): "A+",
    (85,89) : "A",
    (80,84) : "A-",
    (76,79) : "B+",
    (72,75) : "B",
    (68,71) : "B-",
    (64,67) : "C+",
    (60,63) : "C",
    (55,59) : "C-",
    (50,54) : "D",
    (0,49)  : "F"
}

In [4]:
# Interacting with the API is a nightmare
# We need to create hased dictionaries where the keys are indexed from 0
# Read more here: https://community.canvaslms.com/t5/Developers-Group/Link-an-outcome-to-a-rubric-via-the-API/td-p/124545

canvas_rubric = {'title': 'Test Rubrics',
                 'free_form_criteria_comments': 'false'}
criteria = []
total_grades = 0
autograde_grades = 0
UBC_LETTER_GRADES[(0, 0)] =  "No points"
for exercise, rubrics in weights.items():
    for rubric, value in rubrics.items():
        if all(_ not in rubric.lower() for _ in ["spark", "bonus"]):
            if rubric.lower() == 'autograde':
                autograde_grades += value
            else:
                ratings = [{'description': letter, 'points': round(sum(grade_range)/ 2 / 100 * value, 2)} \
                           for grade_range, letter in UBC_LETTER_GRADES.items()]
                ratings = dict(zip(range(len(ratings)), ratings))
                criteria.append({'description': f"{exercise}: {rubric}",
                     'long_description': 'Rubric descriptions can be found here: https://github.com/UBC-MDS/public/tree/master/rubric.',
                     'points': value,
                     'criterion_use_range': False,
                     'ratings': ratings
                    })
            total_grades += value if 'is_bonus' not in rubrics.keys() else 0
# I want to group all my autograded questions into one place which is why I do this, but others can remove this            
if autograde_grades:
    ratings = [{'description': 'Points', 'points': autograde_grades},
               {'description': 'No points', 'points': 0.0}]
    ratings = dict(zip(range(len(ratings)), ratings))
    criteria.append({'description': "Autograded Exercises",
         'long_description': 'Check the autograde score in the assignment.',
         'points': autograde_grades,
         'criterion_use_range': True,
         'ratings': ratings
        })
canvas_rubric['criteria'] = dict(zip(range(len(criteria)), criteria))
# Associate the rubric with the course or a specific assignment
# rubric_association = {'association_id': 53659,
#                       'association_type': 'Course'} # associate rubric with a course (dont link to an assignment)
rubric_association = {'association_id': new_assignment.id,
                      'association_type': 'Assignment',
                      'use_for_grading': 'true',
                      'hide_score_total': 'false',
                      'purpose': 'grading'} # associate with an assignment

NameError: name 'new_assignment' is not defined

In [ ]:
new_rubric = course.create_rubric(rubric=canvas_rubric,
                                  rubric_association=rubric_association)

In [ ]:
updated_assignment = new_assignment.edit(assignment={'points_possible': total_grades})